In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import xgboost as xgb
import numpy as np
from sklearn.metrics import accuracy_score

import matplotlib.pyplot as plt


In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

trainset = torchvision.datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=len(trainset), shuffle=True)

testset = torchvision.datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=len(testset), shuffle=False)


100%|██████████| 26421880/26421880 [00:20<00:00, 1318475.90it/s]


Extracting ./data/FashionMNIST/raw/train-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 299185.50it/s]


Extracting ./data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:04<00:00, 887451.31it/s] 


Extracting ./data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 25791.28it/s]

Extracting ./data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/FashionMNIST/raw



In [6]:
train_data_iter = iter(trainloader)
train_images, train_labels = next(train_data_iter)

test_data_iter = iter(testloader)
test_images, test_labels = next(test_data_iter)

train_images = train_images.view(-1, 28*28).numpy()
train_labels = train_labels.numpy()

test_images = test_images.view(-1, 28*28).numpy()
test_labels = test_labels.numpy()

# Convert labels to 1D array
train_labels = train_labels.ravel()
test_labels = test_labels.ravel()



In [7]:
# plt.imshow(train_images[10].reshape(28, 28))

In [8]:
dtrain = xgb.DMatrix(train_images, label=train_labels)
dtest = xgb.DMatrix(test_images, label=test_labels)

param = {
    'max_depth': 6,
    'eta': 0.1,
    'objective': 'multi:softmax',
    'num_class': 10,
    'eval_metric': 'mlogloss'
}

num_round = 100
evallist  = [(dtrain, 'eval'), (dtest, 'eval')]
bst = xgb.train(param, dtrain, num_round, evals=evallist, verbose_eval=True)

[0]	eval-mlogloss:1.97905
[1]	eval-mlogloss:1.76250
[2]	eval-mlogloss:1.59738
[3]	eval-mlogloss:1.46382
[4]	eval-mlogloss:1.35261
[5]	eval-mlogloss:1.25705
[6]	eval-mlogloss:1.17460
[7]	eval-mlogloss:1.10158
[8]	eval-mlogloss:1.03764
[9]	eval-mlogloss:0.98030
[10]	eval-mlogloss:0.92878
[11]	eval-mlogloss:0.88265
[12]	eval-mlogloss:0.84076
[13]	eval-mlogloss:0.80278
[14]	eval-mlogloss:0.76862
[15]	eval-mlogloss:0.73791
[16]	eval-mlogloss:0.70983
[17]	eval-mlogloss:0.68368
[18]	eval-mlogloss:0.66008
[19]	eval-mlogloss:0.63848
[20]	eval-mlogloss:0.61822
[21]	eval-mlogloss:0.59986
[22]	eval-mlogloss:0.58295
[23]	eval-mlogloss:0.56687
[24]	eval-mlogloss:0.55212
[25]	eval-mlogloss:0.53844
[26]	eval-mlogloss:0.52570
[27]	eval-mlogloss:0.51412
[28]	eval-mlogloss:0.50294
[29]	eval-mlogloss:0.49246
[30]	eval-mlogloss:0.48276
[31]	eval-mlogloss:0.47372
[32]	eval-mlogloss:0.46506
[33]	eval-mlogloss:0.45730
[34]	eval-mlogloss:0.44981
[35]	eval-mlogloss:0.44300
[36]	eval-mlogloss:0.43672
[37]	eval-m

In [10]:
preds = bst.predict(dtest)

accuracy = accuracy_score(test_labels, preds)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 88.35%


In [11]:
preds = bst.predict(dtrain)

accuracy = accuracy_score(train_labels, preds)
print(f'Accuracy: {accuracy * 100:.2f}%')


Accuracy: 96.12%
